# Installation

In [ ]:
!git clone https://github.com/liaorongfan/DeepPersonality.git

In [ ]:
import os
os.chdir("/content/DeepPersonality")
!pip install -r requirements.txt

# Prepare data

In [3]:
!pwd
!gdown 1eANtFzA75Mqv01nW45exfauAIMLNO70J --output ChaLearn2016_tiny_processed.zip

/content/DeepPersonality
Downloading...
From: https://drive.google.com/uc?id=1eANtFzA75Mqv01nW45exfauAIMLNO70J
To: /content/DeepPersonality/ChaLearn2016_tiny_processed.zip
100% 1.59G/1.59G [00:08<00:00, 193MB/s]


In [ ]:
!unzip ChaLearn2016_tiny_processed.zip -d datasets

# Train your own model
If you have designed your own model and want to conduct an experiment on it. Base on the DeepPersonality code base, there are only two steps need to be bone. 
1. Register your model
2. Modify default configration

## Register your model 

The designed model can be `registered` by a `decorater` named `NETWORK_REGISTRY`, shown as below. 
>To acturally run the following demo model, you should create a python script `xx_net.py` and copy the code snap into it. And then `import` the it in `__init__.py` file.

For the demonstration VGG model, it is copied into a file named `demo_model.py` under directory `dpcv/modeling/networks`, and imported in file `dpcv/modeling/networks/__init__.py`

In [ ]:
# dpcv/modeling/networks
# demo_model.py
import torch
import torchvision.models as models
from dpcv.modeling.networks.build import NETWORK_REGISTRY


@NETWORK_REGISTRY.register()
def demo_model(args=None):
    model = models.vgg16(num_classes=5)
    model = model.to(
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )
    return model


In [ ]:
# dpcv/modeling/networks/__init__.py
import dpcv.modeling.networks.demo_model

## Modify default configration

There are different `data_loader` and `modal_trainer` to meet the requirements from different model inputs. Take the demo experiments we used for example, it employ bimodal_renet18 network whch takes audio and visual data as input. While the vgg model only takes visual data as input, we should change the `data_loader` and `modal_trainer` as follows.

In [ ]:
!python script/run_exp.py \
    -c /content/DeepPersonality/config/demo/bimodal_resnet18.yaml \
    --set MODEL.NAME demo_model\
    DATA_LOADER.NAME single_frame_data_loader \
    TRAIN.TRAINER ImageModalTrainer \
    TRAIN.OUTPUT_DIR results/demo_model


And of course we can write those configration into a corresponding `yaml` config file, shown as below. 
```
# config/demo/add_user_model_vgg.yaml
DATA:
  ROOT: "datasets"
  TRAIN_IMG_DATA: "ChaLearn2016_tiny/train_data"
  VALID_IMG_DATA: "ChaLearn2016_tiny/valid_data"
  TEST_IMG_DATA: "ChaLearn2016_tiny/test_data"
  TRAIN_AUD_DATA: "ChaLearn2016_tiny/voice_data/voice_librosa/train_data"
  VALID_AUD_DATA: "ChaLearn2016_tiny/voice_data/voice_librosa/valid_data"
  TEST_AUD_DATA: "ChaLearn2016_tiny/voice_data/voice_librosa/test_data"
  TRAIN_LABEL_DATA: "ChaLearn2016_tiny/annotation/annotation_training.pkl"
  VALID_LABEL_DATA: "ChaLearn2016_tiny/annotation/annotation_validation.pkl"
  TEST_LABEL_DATA: "ChaLearn2016_tiny/annotation/annotation_test.pkl"
DATA_LOADER:
  NAME: "single_frame_data_loader"
  TRANSFORM: "standard_frame_transform"
  TRAIN_BATCH_SIZE: 8
  VALID_BATCH_SIZE: 4
  NUM_WORKERS: 0
MODEL:
  NAME: "demo_model"
LOSS:
  NAME: "mean_square_error"
SOLVER:
  NAME: "sgd"
  SCHEDULER: "multi_step_scale"
  LR_INIT: 0.001
  WEIGHT_DECAY: 0.0005
  MOMENTUM: 0.9
  MILESTONE: [100, 200]
TRAIN:
  TRAINER: "ImageModalTrainer"
  MAX_EPOCH: 30
  OUTPUT_DIR: "results/demo/vgg16"
  ```
  > To run the config file, create it in the code repo first. 

In [6]:
!python script/run_exp.py -c config/demo/add_user_model_vgg.yaml

Valid: Epoch[021/030] Train Mean_Acc: 86.45% Valid Mean_Acc:88.42% OCEAN_ACC:[0.8922451  0.87088394 0.88151866 0.8941389  0.8822897 ]

INFO:log.log:Valid: Epoch[021/030] Train Mean_Acc: 86.45% Valid Mean_Acc:88.42% OCEAN_ACC:[0.8922451  0.87088394 0.88151866 0.8941389  0.8822897 ]

Training: learning rate:0.001
INFO:log.log:Training: learning rate:0.001
Valid: Epoch[022/030] Train Mean_Acc: 85.92% Valid Mean_Acc:88.64% OCEAN_ACC:[0.9040103 0.8766871 0.8838394 0.8911801 0.8765133]

INFO:log.log:Valid: Epoch[022/030] Train Mean_Acc: 85.92% Valid Mean_Acc:88.64% OCEAN_ACC:[0.9040103 0.8766871 0.8838394 0.8911801 0.8765133]

Training: learning rate:0.001
INFO:log.log:Training: learning rate:0.001
Valid: Epoch[023/030] Train Mean_Acc: 86.63% Valid Mean_Acc:87.34% OCEAN_ACC:[0.89718133 0.86010647 0.86875945 0.8755515  0.865557  ]

INFO:log.log:Valid: Epoch[023/030] Train Mean_Acc: 86.63% Valid Mean_Acc:87.34% OCEAN_ACC:[0.89718133 0.86010647 0.86875945 0.8755515  0.865557  ]

Training: learn